In [ ]:
raise RuntimeError(
    """
    Execution Locked: This notebook has already been run.
    Running this notebook will delete all progress.
    To unlock, comment out this cell
    """
)

In [ ]:
import torch
import difflib
import logging
import itertools
import pandas as pd
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from transformers import AutoModelForCausalLM, AutoTokenizer

# Definitions

## Model

In [ ]:
model_name_or_path = "./models/DeepSeek-R1-Distill-Qwen-1.5B"
# Load your model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.float16, device_map="auto"
)


## Test Prompt and Expected Results

In [10]:
test_prompt = f"""
    <think>\n Perform the following:
    Create a cohesive story using this student data (JSON) for an Academic Advisor's analysis:
    '''
    'student': 'Andy', 'grade': 'A', 'course': 'Calculus II', 'student_notes': 'Student performs well in previous mathematics courses.'
    '''
    Each key-value pair in the JSON should strictly represent 1 sentence that would strictly create 1 sentence in the cohesive story.
    The number of key-value pairs is strictly equivalent to the number of sentences in the cohesive story.
    Limit the cohesive story to only and exactly 4 sentences using the JSON data.
    """

expected_results = f"""
    Alright, so I have this JSON data about Andy, his grade, the course he's taking, and some notes from his professor. I need to create a cohesive story using exactly four sentences. Each key-value pair in the JSON should correspond to one sentence. Let me break it down step by step.

    First, the JSON has four key-value pairs: student, grade, course, and student_notes. That means I need four sentences in total. Each sentence should cover one of these points without overlapping.

    Starting with the student's name: Andy. That's straightforward. I can introduce Andy as a student.

    Next, his grade is an 'A'. So the second sentence could be about his performance. Maybe something like, "Andy has demonstrated a strong command of the material in his Calculus II course."

    Then, the course he's taking is Calculus II. So the third sentence should mention that he's in this specific course. Maybe, "He is currently enrolled in Calculus II, building upon his previous mathematical foundation."

    Lastly, the student_notes say he performs well in previous math courses. That should be the fourth sentence. Perhaps, "His prior achievements in mathematics courses have prepared him well for this advanced level."

    Putting it all together, I have four sentences that flow naturally, each corresponding to a JSON key. It feels cohesive, each part connecting smoothly to the next. I think this works!
    </think>

    Andy has demonstrated a strong command of the material in his Calculus II course. He is currently enrolled in Calculus II, building upon his previous mathematical foundation. His prior achievements in mathematics courses have prepared him well for this advanced level.<｜end▁of▁sentence｜>
"""

## Generate Text

In [11]:
def generate_text(model, tokenizer, prompt, top_p, top_k, temperature):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    input_data = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096,
    )
    input_ids = input_data.input_ids.to(device)

    attention_mask = input_data.attention_mask.to(device)

    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        do_sample=True,  # Ensure sampling since testing sampling-based hyperparameters
        repetition_penalty=1.2,  # Helps reduce looping issues
        pad_token_id=tokenizer.eos_token_id,
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

## Logging

In [20]:
log_filename = "bayesian_search_log.txt"
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format="%(asctime)s - top_p=%(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# Grid Search

In [34]:
# Define search space
top_p_values = [0.9, 0.95, 1.0]
top_k_values = [40, 45, 50]
temperature_values = [0.5, 0.6, 0.7]


hyperparameter_combinations = list(
    itertools.product(top_p_values, top_k_values, temperature_values)
)

## Grid Search - Scoring

In [35]:
grid_search_results = []

# Run grid search
for top_p, top_k, temperature in hyperparameter_combinations:
    output_text = generate_text(
        model, tokenizer, test_prompt, top_p, top_k, temperature
    )

    similarity_score = difflib.SequenceMatcher(
        None, expected_results, output_text
    ).ratio()

    grid_search_results.append(
        {
            "top_p": top_p,
            "top_k": top_k,
            "temperature": temperature,
            "similarity": similarity_score,
            "output": output_text,
        }
    )

### Grid Search - Results

In [36]:
# Convert results to a dataframe and save
df = pd.DataFrame(grid_search_results)
df.to_csv("grid_search_results.csv", index=False)
df = df.drop(columns=["output"])
df = df.sort_values(by="similarity", ascending=False)

df

,top_p,top_k,temperature,similarity
5,0.90,45,0.7,0.095238
23,1.00,45,0.7,0.094693
11,0.95,40,0.7,0.093761
14,0.95,45,0.7,0.087906
1,0.90,40,0.6,0.085490
19,1.00,40,0.6,0.082487
21,1.00,45,0.5,0.082274
12,0.95,45,0.5,0.082152
15,0.95,50,0.5,0.080443
25,1.00,50,0.6,0.080405


# Bayesian Search

In [38]:
search_space = [
    Real(0.97, 1.0, name="top_p"),
    Integer(40, 50, name="top_k"),
    Real(0.5, 0.7, name="temperature"),
]

## Bayesian Search - Scoring

In [39]:
@use_named_args(search_space)
def objective(top_p, top_k, temperature):
    top_k = int(top_k)
    output_text = generate_text(
        model, tokenizer, test_prompt, top_p, top_k, temperature
    )

    similarity_score = difflib.SequenceMatcher(
        None, expected_results, output_text
    ).ratio()

    logging.info(
        f"Tested: top_p={top_p}, top_k={top_k}, temperature={temperature} -> similarity={similarity_score:.4f}"
    )

    return -similarity_score


bayesian_result = gp_minimize(
    objective, search_space, n_calls=200, random_state=42, verbose=True
)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 21.0023
Function value obtained: -0.0834
Current minimum: -0.0834
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 24.1834
Function value obtained: -0.0797
Current minimum: -0.0834
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 126.1004
Function value obtained: -0.0609
Current minimum: -0.0834
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 59.5000
Function value obtained: -0.0843
Current minimum: -0.0843
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 211.4444
Function value obtained: -0.0672
Current minimum: -0.0843
Iteration No: 6 st

### Bayesian Search - Results

In [40]:
best_top_p, best_top_k, best_temperature = bayesian_result.x
best_similarity_score = -bayesian_result.fun

best_performance = f"""
Best Hyperparameters Found in Bayesian Search:\n
top_p: {best_top_p}
top_k: {best_top_k}
temperature: {best_temperature}
similarity: {best_similarity_score}
"""
logging.info(best_performance)
print(best_performance)


Best Hyperparameters Found in Bayesian Search:

top_p: 0.9929229130165201
top_k: 42
temperature: 0.6267068339449485
similarity: 0.11118930330752991



# End Result for *top_p, top_k,* and *temperature* hyperparameters

In [ ]:
final_top_p = 0.99
final_top_k = 42
final_temperature = 0.63